In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.applications.vgg16 import VGG16

from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

2023-04-12 17:46:05.826823: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Modelo valor

In [43]:
# Charge model VGC16 with imagenet weights
model_vgc = VGG16(weights='imagenet', include_top=False, input_shape=(224, 150, 3))
for(layer) in model_vgc.layers:
    layer.trainable = False

x = model_vgc.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(13, activation='softmax')(x)  

model = Model(inputs=model_vgc.input, outputs=x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 75, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 75, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 75, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 37, 128)       0   

In [44]:
folder = 'images/training/labeled/'
from keras.applications.vgg16 import preprocess_input

# Data generator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True, validation_split=0.2)
train_generator = datagen.flow_from_directory(folder, target_size=(224, 150), batch_size=13, class_mode='categorical', subset='training')
test_generator = datagen.flow_from_directory(folder, target_size=(224, 150), batch_size=13, class_mode='categorical', subset='validation')

print(test_generator.class_indices)


Found 65 images belonging to 13 classes.
Found 13 images belonging to 13 classes.
{'10': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, 'A': 9, 'J': 10, 'K': 11, 'Q': 12}


In [45]:
history = model.fit_generator(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10


/var/folders/vb/vzpmbxtd0j3gyd335m8bdw_h0000gn/T/ipykernel_14070/3671890123.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=10, validation_data=test_generator)


5/5 [==============================] - 12s 2s/step - loss: 48.6038 - accuracy: 0.3385 - val_loss: 52.6176 - val_accuracy: 0.3077
Epoch 2/10
5/5 [==============================] - 12s 3s/step - loss: 23.1754 - accuracy: 0.6615 - val_loss: 8.2284 - val_accuracy: 0.7692
Epoch 3/10
5/5 [==============================] - 10s 2s/step - loss: 4.4006 - accuracy: 0.9077 - val_loss: 1.4500 - val_accuracy: 0.9231
Epoch 4/10
5/5 [==============================] - 12s 3s/step - loss: 0.9895 - accuracy: 0.9692 - val_loss: 1.7791 - val_accuracy: 0.9231
Epoch 5/10
5/5 [==============================] - 12s 2s/step - loss: 0.5066 - accuracy: 0.9692 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 10s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 11s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 8/10
5/5 [======

In [47]:
model.save('model_value_1024.h5')

In [48]:
def decode_predictions_custom(preds, top=3, class_list=None):
    if class_list is None:
        class_list = ['10','2', '3', '4', '5', '6', '7', '8', '9', 'A', 'J', 'K', 'Q']
        
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [(class_list[i], pred[i]*100.0) for i in top_indices]
        results.append(result)
    return results

In [60]:
import cv2
import numpy as np

from tensorflow.keras.preprocessing import image

var = cv2.imread('Prueba7.png')
var_resized = cv2.resize(var, (150, 224))

var_resized = image.img_to_array(var_resized)
var_resized = np.expand_dims(var_resized, axis=0)
var_resized = preprocess_input(var_resized)
pretiction = model.predict(var_resized)

decoded = decode_predictions_custom(pretiction)
print(decoded)


1/1 [==============================] - 0s 142ms/step
[[('2', 99.99998807907104), ('3', 1.279527168662753e-05), ('Q', 0.0)]]
